In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
(X_train_full, y_train_full), (X_test,y_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
X_train_full.shape

(60000, 28, 28)

In [5]:
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

In [6]:
X_test = X_test[:5000]
y_test = y_test[:5000]

In [7]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [8]:
X_valid,X_train = X_train_full[:5000],X_train_full[5000:]

In [9]:
y_valid,y_train = y_train_full[:5000],y_train_full[5000:]

In [10]:
def split_dataset(X,y):
    y_5_or_6 = (y==5)|(y==6)
    y_A = y[~y_5_or_6]
    y_A[y_A>6] -= 2
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [11]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)


In [12]:
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)


In [13]:
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)


In [14]:
tf.random.set_seed(42)
np.random.seed(42)

In [15]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_A.add(keras.layers.Dense(n_hidden,activation="selu"))
model_A.add(keras.layers.Dense(8,activation="softmax"))

In [16]:
model_A.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.SGD(1e-3),metrics=["accuracy"])

In [17]:
history = model_A.fit(X_train_A, y_train_A, epochs=5,
            validation_data=(X_valid_A, y_valid_A))

Epoch 1/5
622/622 [==============================] - 2s 3ms/step - loss: 0.7979 - accuracy: 0.7498 - val_loss: 0.4762 - val_accuracy: 0.8490
Epoch 2/5
622/622 [==============================] - 2s 3ms/step - loss: 0.4260 - accuracy: 0.8582 - val_loss: 0.3921 - val_accuracy: 0.8655
Epoch 3/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3702 - accuracy: 0.8736 - val_loss: 0.3703 - val_accuracy: 0.8710
Epoch 4/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3418 - accuracy: 0.8810 - val_loss: 0.3285 - val_accuracy: 0.8876
Epoch 5/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3240 - accuracy: 0.8870 - val_loss: 0.3230 - val_accuracy: 0.8879


In [18]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [19]:
model_A.save("my_model_A2.h5")

In [20]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="softmax"))

In [21]:
model_B.compile(loss = "binary_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])

/usr/local/anaconda/envs/tensorflow2/lib/python3.6/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [22]:
history = model_B.fit(X_train_B, y_train_B, epochs=5,
            validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 4ms/step - loss: 0.2678 - accuracy: 0.5050 - val_loss: 0.1301 - val_accuracy: 0.5000
Epoch 2/5
161/161 [==============================] - 0s 3ms/step - loss: 0.0910 - accuracy: 0.5050 - val_loss: 0.0765 - val_accuracy: 0.5000
Epoch 3/5
161/161 [==============================] - 0s 3ms/step - loss: 0.0573 - accuracy: 0.5050 - val_loss: 0.0565 - val_accuracy: 0.5000
Epoch 4/5
161/161 [==============================] - 0s 3ms/step - loss: 0.0423 - accuracy: 0.5050 - val_loss: 0.0456 - val_accuracy: 0.5000
Epoch 5/5
161/161 [==============================] - 0s 3ms/step - loss: 0.0335 - accuracy: 0.5050 - val_loss: 0.0389 - val_accuracy: 0.5000


In [23]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [24]:
model_A_clone = keras.models.clone_model(model_A)

set to set model parameters and get to get those weigths baby

In [25]:
model_A_clone.set_weights(model_A.get_weights())

Now, create a new model model_B_on_A, based on existing layers of model_A.

In [27]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

Set all the layers, except the last layer, of model_B_on_A to be non-trainable.

In [28]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable=False

In [29]:
model_B_on_A.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
Total params: 275,750
Trainable params: 2,550
Non-trainable params: 273,200
____________________________________________

In [30]:
model_B_on_A.compile(loss="binary_crossentropy",
         optimizer=keras.optimizers.SGD(lr=1e-3),
         metrics=["accuracy"])

In [31]:
history = model_B_on_A.fit(X_train_B,y_train_B,epochs=5,validation_data=(X_valid_B,y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 3ms/step - loss: 4.2350 - accuracy: 0.0020 - val_loss: 3.7094 - val_accuracy: 0.0000e+00
Epoch 2/5
161/161 [==============================] - 0s 2ms/step - loss: 3.2732 - accuracy: 0.0027 - val_loss: 2.9474 - val_accuracy: 0.0000e+00
Epoch 3/5
161/161 [==============================] - 0s 2ms/step - loss: 2.6099 - accuracy: 0.0016 - val_loss: 2.3775 - val_accuracy: 0.0010
Epoch 4/5
161/161 [==============================] - 0s 2ms/step - loss: 2.1167 - accuracy: 0.0021 - val_loss: 1.9802 - val_accuracy: 0.0010
Epoch 5/5
161/161 [==============================] - 0s 2ms/step - loss: 1.7152 - accuracy: 0.0031 - val_loss: 1.5892 - val_accuracy: 0.0010


In [34]:
model_B.evaluate(X_test_B, y_test_B)

31/31 [==============================] - 0s 1ms/step - loss: 0.0319 - accuracy: 0.4984


[0.03187720105051994, 0.49844881892204285]

In [35]:
model_B_on_A.evaluate(X_test_B,y_test_B)

31/31 [==============================] - 0s 2ms/step - loss: 1.5118 - accuracy: 0.0052


[1.5117892026901245, 0.005170630756765604]